# Fotmob Seasonwide Shotdata Scraper


This Scraper works using Selenium so make sure to download Chromedrivers (reccomend to use chrome).<br>
Link to download: https://chromedriver.chromium.org/downloads ( before download do check your Chrome version)
<br>
Note: This scraper works for Current season data for All T5 Leagues. For Previous seasons data 
it will be eventually added & code will see modfications, until then enjoy latest Shots data with x,y coordinates & xG values!!

In [1]:
#Import Libraries
import pandas as pd
import mplsoccer as mps
import pickle
import matplotlib.pyplot as plt
from mplsoccer import add_image
from PIL import Image
from matplotlib.colors import to_rgba
from urllib.request import urlopen
import time
import time
import pandas as pd
pd.options.mode.chained_assignment = None
import json
from bs4 import BeautifulSoup as soup
import re 
from collections import OrderedDict
import datetime
from datetime import datetime as dt
import itertools
import numpy as np
import pickle
try:
    from tqdm import trange
except ModuleNotFoundError:
    pass

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import difflib
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [17]:
pd.set_option("display.max_columns",300)
pd.set_option("display.max_rows",500)

In [2]:
# T5 League URLS
Premier_League='https://www.fotmob.com/leagues/47/overview/premier-league'
Serie_A='https://www.fotmob.com/leagues/55/overview/serie-a'
LaLiga='https://www.fotmob.com/leagues/87/overview/laliga'
Bundesliga='https://www.fotmob.com/leagues/54/overview/bundesliga'
Ligue1='https://www.fotmob.com/leagues/53/overview/ligue-1'

In [3]:
# Returns a Dataframe containing Home_team,away_team, Matchid & Fotmob Match URL of the game
def FOTMOB_Leaguegames_Urls(main_url):
    list1=[]
    list2=[]
    yes=True
    driver = webdriver.Chrome("C:/Users/Rahul/Pictures/Juve Event Data/chromedriver_win32/chromedriver.exe")
    driver.maximize_window()
    driver.get(main_url)
    try:
        driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/button').click()
    except:
        print('No cookies button so we move')
    driver.find_element_by_xpath('/html/body/div[1]/main/section/div/section/nav/a[4]').click()
    y=driver.current_url
    while(yes==True):
        driver = webdriver.Chrome("C:/Users/Rahul/Pictures/Juve Event Data/chromedriver_win32/chromedriver.exe")
        driver.get(y)
        try:
            driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/button').click()
        except:
            print('No cookies button so we move')
        driver.find_element_by_xpath('//*[@id="__next"]/main/section/div[2]/section/div/button[1]').click()
        y=driver.current_url
        driver.close()
        list1.append(y)
        if(y[-2:]=='=0'):
            yes=False
    for i in range(len(list1)):
            driver = webdriver.Chrome("C:/Users/Rahul/Pictures/Juve Event Data/chromedriver_win32/chromedriver.exe")
            driver.maximize_window()
            driver.get(list1[i])
            time.sleep(5)
            x=driver.find_element_by_xpath('/html/body/div[1]/main/section/div[2]/section').get_attribute('outerHTML').split(sep='<')
            for i in range(len(x)):
                if(i==0):
                    continue
                if(x[i][0]=='a'):
                    list2.append(x[i].split(sep='"')[1])  
    df=pd.DataFrame()
    df['Home_Team']=[]
    df['Away_Team']=[]
    df['MatchId']=[]
    for i in range(len(list2)):
        k=[list2[i].split(sep='/')[4].split(sep='-vs-')[0],list2[i].split(sep='/')[4].split(sep='-vs-')[1],list2[i].split(sep='/')[2]]
        k1={'Home_Team':k[0],'Away_Team':k[1],'MatchId':k[2]}
        df = df.append(k1, ignore_index = True)
    df['match_urls']=list2    
    return df    

In [4]:
# Using Above Function to get Premier League Matches Data (This function notoriously takes time which I am working on)
Matches=FOTMOB_Leaguegames_Urls(Premier_League)

No cookies button so we move


In [5]:
# A DataFrame with all Matches, Id's & URL's
Matches

,Home_Team,Away_Team,MatchId,match_urls
0,afc-bournemouth,chelsea,3901272,/match/3901272/matchfacts/afc-bournemouth-vs-c...
1,manchester-city,leeds-united,3901276,/match/3901276/matchfacts/manchester-city-vs-l...
2,tottenham-hotspur,crystal-palace,3901279,/match/3901279/matchfacts/tottenham-hotspur-vs...
3,wolverhampton-wanderers,aston-villa,3901281,/match/3901281/matchfacts/wolverhampton-wander...
4,liverpool,brentford,3901275,/match/3901275/matchfacts/liverpool-vs-brentford
...,...,...,...,...
342,tottenham-hotspur,southampton,3900938,/match/3900938/matchfacts/tottenham-hotspur-vs...
343,everton,chelsea,3900939,/match/3900939/matchfacts/everton-vs-chelsea
344,leicester-city,brentford,3900936,/match/3900936/matchfacts/leicester-city-vs-br...
345,manchester-united,brighton--hove-albion,3900940,/match/3900940/matchfacts/manchester-united-vs...


In [6]:
# This Function has been inspired from Chaitanya98 on Github:'https://github.com/Chaitanya98/Barcalytix/blob/main/Files%20%26%20Python%20Notebooks/FOTMOB_API.py'
import requests
def getFOTMOBShots(url, returnJSON=False):
        
    # Extracting match ID from match URL
    matchLink = url
    matchId = matchLink.split("/")[4]

    apiLink = "https://www.fotmob.com/api/matchDetails?matchId=" + matchId

    # Making request to API and loading JSON response
    response = requests.get(apiLink)
    data = json.loads(response.content)

    # Creating a pandas DataFrame from shot data    
    shotsData = pd.DataFrame(data["content"]["shotmap"]["shots"])

    # Extract the home team and away team data from the main data dictionary
    homeTeam = data['general']['homeTeam']
    awayTeam = data['general']['awayTeam']

    # Create a DataFrame to store the team names and join it with the shots data
    teamNames = pd.DataFrame([homeTeam] + [awayTeam])
    shotsData = shotsData.join(teamNames.set_index("id"), on="teamId")
    
    # Rename the `name` column to teamName
    shotsData.rename(columns={'name':'teamName'}, inplace=True)

    # Create new columns for the home team and away team names
    shotsData["homeTeam"] = homeTeam["name"]
    shotsData["awayTeam"] = awayTeam["name"]

    # Create a new column to indicate whether the shot was taken by the home or away team
    shotsData['h_a'] = np.where(shotsData["teamId"] == homeTeam["id"], 'home', 'away')

    # Update the 'eventType' column to 'Blocked' for blocked shots
    shotsData.loc[shotsData['isBlocked'], 'eventType'] = 'Blocked'

    # Calculate the distance from goal
    shotsData['distanceFromGoal'] = np.sqrt((105 - shotsData['x']) ** 2 + (34 - shotsData['y']) ** 2)
    
    # Extract the 'x' and 'y' values from the 'onGoalShot' column
    shotsData['onGoalX'] = shotsData['onGoalShot'].str['x']/2
    shotsData['onGoalY'] = shotsData['onGoalShot'].str['y']/(2/3)
    
    if returnJSON:
        return shotsData, data
    else:        
        return shotsData

In [15]:
# Get Shots data for All Brighton matches for Example (enter the team you want by checking the Matches DataFrame)
Matches=Matches[(Matches['Home_Team']=='brighton--hove-albion')|(Matches['Home_Team']=='brighton--hove-albion')]
shotsdf=pd.DataFrame()
for i in range(len(Matches)):
    df=getFOTMOBShots('https://www.fotmob.com' + Matches.iloc[i]['match_urls'])
    shotsdf=shotsdf.append(getFOTMOBShots('https://www.fotmob.com' + Matches.iloc[i]['match_urls']))    
shotsdf=shotsdf.reset_index()                

In [18]:
# Here's your favorite Team's Shot data for whole season
shotsdf

,index,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,isOnTarget,blockedX,blockedY,goalCrossedY,goalCrossedZ,expectedGoals,expectedGoalsOnTarget,shotType,situation,period,isOwnGoal,onGoalShot,firstName,lastName,fullName,teamColor,shortName,teamName,homeTeam,awayTeam,h_a,distanceFromGoal,onGoalX,onGoalY
0,0,2545938205,Goal,8668,352887,Abdoulaye Doucouré,98.631579,36.058750,1,NaN,False,True,NaN,NaN,36.058750,0.770526,0.314717,0.7087,RightFoot,RegularPlay,FirstHalf,False,"{'x': 0.455357142857142, 'y': 0.20384293968253...",Abdoulaye,Doucouré,Abdoulaye Doucouré,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,6.692924,2.276786e-01,0.305764
1,1,2545939613,Miss,10204,113836,Danny Welbeck,96.315789,32.627500,4,NaN,False,False,NaN,NaN,24.955625,0.802632,0.164898,NaN,Header,RegularPlay,FirstHalf,False,"{'x': 2, 'y': 0.08874373021905878, 'zoomRatio'...",Danny,Welbeck,Danny Welbeck,#0850A0,NaN,Brighton & Hove Albion,Brighton & Hove Albion,Everton,home,8.792000,1.000000e+00,0.133116
2,2,2545943733,Miss,10204,167008,Pascal Gross,83.498544,20.496051,13,NaN,False,False,95.736842,31.788750,42.697500,0.532947,0.037648,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': 0, 'y': 0.06127592593273932, 'zoomRatio'...",Pascal,Gross,Pascal Gross,#0850A0,NaN,Brighton & Hove Albion,Brighton & Hove Albion,Everton,home,25.390338,0.000000e+00,0.091914
3,3,2545944171,Blocked,10204,831489,Alexis Mac Allister,94.289474,40.805714,14,NaN,True,True,96.026316,40.301429,35.753750,1.220000,0.071781,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': 0.5360449735449745, 'y': 0.3227513211640...",Alexis,Mac Allister,Alexis Mac Allister,#0850A0,Mac Allister,Brighton & Hove Albion,Brighton & Hove Albion,Everton,home,12.689883,2.680225e-01,0.484127
4,4,2545945605,Miss,8668,612150,Dominic Calvert-Lewin,86.997059,28.791190,16,NaN,False,False,NaN,NaN,41.141905,0.532947,0.045237,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': -2.220446049250313e-16, 'y': 0.074622580...",Dominic,Calvert-Lewin,Dominic Calvert-Lewin,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,18.741334,-1.110223e-16,0.111934
5,5,2545947667,Blocked,8668,939542,Dwight McNeil,91.400000,44.981656,24,NaN,True,True,92.600000,43.930701,34.305000,1.220000,0.037538,NaN,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 0.9193121693121694, 'y': 0.3227513211640...",Dwight,McNeil,Dwight McNeil,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,17.480182,4.596561e-01,0.484127
6,6,2545947831,Blocked,8668,179410,Idrissa Gana Gueye,88.600000,23.018344,24,NaN,True,True,93.600000,26.285312,35.220000,1.220000,0.028573,NaN,RightFoot,FromCorner,FirstHalf,False,"{'x': 0.6772486772486757, 'y': 0.3227513211640...",Idrissa Gana,Gueye,Idrissa Gana Gueye,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,19.737193,3.386243e-01,0.484127
7,7,2545948143,Blocked,8668,939542,Dwight McNeil,94.289474,30.492500,25,NaN,True,True,96.508772,31.407500,34.838750,1.220000,0.183389,NaN,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 0.7781084656084645, 'y': 0.3227513211640...",Dwight,McNeil,Dwight McNeil,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,11.270223,3.890542e-01,0.484127
8,8,2545949221,Goal,8668,352887,Abdoulaye Doucouré,94.289474,24.489682,29,NaN,False,True,NaN,NaN,31.178750,0.976000,0.174123,0.6255,RightFoot,FastBreak,FirstHalf,False,"{'x': 1.7463624338624355, 'y': 0.2582010569312...",Abdoulaye,Doucouré,Abdoulaye Doucouré,#00369c,NaN,Everton,Brighton & Hove Albion,Everton,away,14.323461,8.731812e-01,0.387302
9,9,2545952415,Goal,10204,107917,Jason Steele,2.465517,31.712500,35,NaN,False,True,NaN,NaN,35.830000,1.014526,NaN,NaN,RightFoot,RegularPlay,FirstHalf,True,"{'x': 0.5158730158730164, 'y': 0.2683932039153...",Jason,Steele,Jason Steele,#0850A0,NaN,Brighton & Hove Albion,Brighton & Hove Albion,Everton,home,102.559996,2.579365e-01,0.402590
